# ResNet50 Image Classification using ONNX and AzureML

This example shows how to deploy the ResNet50 ONNX model as a web service using Azure Machine Learning services and the ONNX Runtime.

## What is ONNX
ONNX is an open format for representing machine learning and deep learning models. ONNX enables open and interoperable AI by enabling data scientists and developers to use the tools of their choice without worrying about lock-in and flexibility to deploy to a variety of platforms. ONNX is developed and supported by a community of partners including Microsoft, Facebook, and Amazon. For more information, explore the [ONNX website](http://onnx.ai).

## ResNet50 Details
ResNet classifies the major object in an input image into a set of 1000 pre-defined classes. For more information about the ResNet50 model and how it was created can be found on the [ONNX Model Zoo github](https://github.com/onnx/models/tree/master/vision/classification/resnet). 

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

## Prerequisites

To make the best use of your time, make sure you have done the following:

* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning
* If you are using an Azure Machine Learning Notebook VM, you are all set.  Otherwise, go through the [configuration notebook](../../../configuration.ipynb) to:
    * install the AML SDK
    * create a workspace and its configuration file (config.json)

In [1]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [2]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.15.0


#### Download pre-trained ONNX model from ONNX Model Zoo.

Download the [ResNet50v2 model and test data](https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet50v2/resnet50v2.tar.gz) and extract it in the same folder as this tutorial notebook.


In [3]:
import urllib.request

onnx_model_url = "https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet50v2/resnet50v2.tar.gz"
urllib.request.urlretrieve(onnx_model_url, filename="resnet50v2.tar.gz")

!tar xvzf resnet50v2.tar.gz

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
resnet50v2/
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
resnet50v2/._resnet50v2.onnx
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
resnet50v2/resnet50v2.onnx
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'


## Deploying as a web service with Azure ML

### Load your Azure ML workspace

We begin by instantiating a workspace object from the existing workspace created earlier in the configuration notebook.

In [4]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = '\n')

workshoplcl
westeurope
workshoplcl-rg


### Register your model with Azure ML

Now we upload the model and register it in the workspace.

In [5]:
from azureml.core.model import Model

model = Model.register(model_path = "resnet50v2/resnet50v2.onnx",
                       model_name = "resnet50v2",
                       tags = {"onnx": "demo"},
                       description = "ResNet50v2 from ONNX Model Zoo",
                       workspace = ws)

Registering model resnet50v2


#### Displaying your registered models

You can optionally list out all the models that you have registered in this workspace.

In [6]:
models = ws.models
for name, m in models.items():
    print("Name:", name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

Name: monmodele 	Version: 1 	Description: None {'type': 'classification', 'framework': 'scikit', 'description': 'madescription', 'run_id': 'monexperimentation_1602575538_84ebca82'}
Name: DiabetesRidge 	Version: 2 	Description: Modèle de régression Ridge {'area': 'Diabetes', 'type': 'Regression Ridge', 'k': '0.2', 'MSE': '3026.667734128127', 'R2': '0.4067092393506434', 'RMSE': '55.015159130262695', 'Framework': 'Azure ML SDK'}
Name: mybestmodel 	Version: 1 	Description: None {'Training context': 'Azure Auto ML'}
Name: local_deploy_model 	Version: 1 	Description: None {}
Name: IBM_attrition_explainer 	Version: 1 	Description: None {}
Name: diabetes_model 	Version: 1 	Description: None {'Training context': 'Pipeline'}
Name: resnet50v2 	Version: 1 	Description: ResNet50v2 from ONNX Model Zoo {'onnx': 'demo'}


### Write scoring file

We are now going to deploy our ONNX model on Azure ML using the ONNX Runtime. We begin by writing a score.py file that will be invoked by the web service call. The `init()` function is called once when the container is started so we load the model using the ONNX Runtime into a global session object.

In [7]:
%%writefile score.py
import json
import time
import sys
import os
import numpy as np    # we're going to use numpy to process input and output data
import onnxruntime    # to inference ONNX models, we use the ONNX Runtime

def softmax(x):
    x = x.reshape(-1)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def init():
    global session
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'resnet50v2.onnx')
    session = onnxruntime.InferenceSession(model, None)

def preprocess(input_data_json):
    # convert the JSON data into the tensor input
    img_data = np.array(json.loads(input_data_json)['data']).astype('float32')
    
    #normalize
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]

    return norm_img_data

def postprocess(result):
    return softmax(np.array(result)).tolist()

def run(input_data_json):
    try:
        start = time.time()
        # load in our data which is expected as NCHW 224x224 image
        input_data = preprocess(input_data_json)
        input_name = session.get_inputs()[0].name  # get the id of the first input of the model   
        result = session.run([], {input_name: input_data})
        end = time.time()     # stop timer
        return {"result": postprocess(result),
                "time": end - start}
    except Exception as e:
        result = str(e)
        return {"error": result}

Writing score.py


### Create inference configuration

First we create a YAML file that specifies which dependencies we would like to see in our container.

In [8]:
from azureml.core.conda_dependencies import CondaDependencies 


myenv = CondaDependencies.create(pip_packages=["numpy", "onnxruntime", "azureml-core", "azureml-defaults"])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Create the inference configuration object. Please note that you must indicate azureml-defaults with verion >= 1.0.45 as a pip dependency, because it contains the functionality needed to host the model as a web service.

In [9]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

### Deploy the model

In [10]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'demo': 'onnx'}, 
                                               description = 'web service for ResNet50 ONNX model')

The following cell will likely take a few minutes to run as well.

In [11]:
%%time
from random import randint

aci_service_name = 'onnx-demo-resnet50'+str(randint(0,100))
print("Service", aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Service onnx-demo-resnet5056
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 937 ms, sys: 142 ms, total: 1.08 s
Wall time: 8min 32s


In case the deployment fails, you can check the logs. Make sure to delete your aci_service before trying again.

In [12]:
if aci_service.state != 'Healthy':
    # run this command for debugging.
    print(aci_service.get_logs())
    aci_service.delete()

## Success!

If you've made it this far, you've deployed a working web service that does image classification using an ONNX model. You can get the URL for the webservice with the code below.

In [13]:
print(aci_service.scoring_uri)

http://434916fa-5dfb-4e74-958a-3a356ba24c31.westeurope.azurecontainer.io/score


When you are eventually done using the web service, remember to delete it.

In [14]:
aci_service.delete()